In [1]:
import random
import numpy as np
import os
import torch
import json
from PIL import Image
from src.env.env import RILAB_OMY_ENV
from torchvision import transforms

from lerobot.common.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
from lerobot.common.datasets.utils import dataset_to_policy_features
from lerobot.configs.types import FeatureType
import glfw

from src.policies.baseline.modeling import BaselinePolicy

/home/rilab/ros2_ws/src/deep_learning_proj/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''
Configuration
'''
ROOT = "./dataset/transformed_data2"
device = 'cuda'
CKPT = "./ckpt/baseline_model_image"

# Environment Configuration
action_type = 'joint'  # Options: 'joint','delta_joint, 'delta_eef_pose', 'eef_pose'
proprio_type = 'joint_pos' # Options: 'joint_pos', 'eef_pose'
observation_type = 'image' # options: 'image', 'object_pose'


# Evaluation Configuration
TEST_EPISODES = 20
MAX_EPISODE_STEPS = 10_000
IMAGE_NOISE = True


## Load Policy

In [3]:
'''
Meta data is for loading dataset statistics and feature information
'''
dataset_metadata = LeRobotDatasetMetadata("transformed_data", root=ROOT)
features = dataset_to_policy_features(dataset_metadata.features)
output_features = {key: ft for key, ft in features.items() if ft.type is FeatureType.ACTION}
input_features = {key: ft for key, ft in features.items() if key not in output_features}

kwargs = {}
kwargs["dataset_stats"] = dataset_metadata.stats
policy = BaselinePolicy.from_pretrained(CKPT, **kwargs)
policy.to('cuda')

Using concatenated features without projection: 1536
Loading weights from local directory


BaselinePolicy(
  (normalize_inputs): Normalize(
    (buffer_observation_state): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
    )
  )
  (normalize_targets): Normalize(
    (buffer_action): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
    )
  )
  (unnormalize_outputs): Unnormalize(
    (buffer_action): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 7 (cuda:0)]
    )
  )
  (model): BaselineModel(
    (visual_encoder): VisualEncoder(
      (model): DINOv3ViTModel(
        (embeddings): DINOv3ViTEmbeddings(
          (patch_embeddings): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (r

## Load Envirionment

In [4]:
'''
Load environment configuration and initialize environments
'''
config_file_path = './configs/train2.json'
with open(config_file_path) as f:
    env_conf = json.load(f)
omy_env = RILAB_OMY_ENV(cfg=env_conf, seed=0, 
                        action_type=action_type, 
                        obs_type=proprio_type,
                        vis_mode = 'teleop')


-----------------------------------------------------------------------------
name:[tabletop_env] dt:[0.002] HZ:[500]
 n_qpos:[34] n_qvel:[31] n_qacc:[31] n_ctrl:[8]
 integrator:[IMPLICITFAST]

n_body:[25]
 [0/25] [world] mass:[0.00]kg
 [1/25] [front_object_table] mass:[1.00]kg
 [2/25] [camera] mass:[0.00]kg
 [3/25] [camera2] mass:[0.00]kg
 [4/25] [camera3] mass:[0.00]kg
 [5/25] [base] mass:[3.59]kg
 [6/25] [link1] mass:[2.06]kg
 [7/25] [link2] mass:[3.68]kg
 [8/25] [link3] mass:[2.39]kg
 [9/25] [link4] mass:[1.40]kg
 [10/25] [link5] mass:[1.40]kg
 [11/25] [link6] mass:[0.65]kg
 [12/25] [camera_center] mass:[0.00]kg
 [13/25] [tcp_link] mass:[0.32]kg
 [14/25] [rh_p12_rn_r1] mass:[0.07]kg
 [15/25] [rh_p12_rn_r2] mass:[0.02]kg
 [16/25] [rh_p12_rn_l1] mass:[0.07]kg
 [17/25] [rh_p12_rn_l2] mass:[0.02]kg
 [18/25] [body_obj_wooden_cabinet] mass:[2.24]kg
 [19/25] [body_obj_wooden_cabinet_top] mass:[3.00]kg
 [20/25] [body_obj_wooden_cabinet_middle] mass:[3.00]kg
 [21/25] [body_obj_wooden_cabin

In [5]:
def get_default_transform():
    """
    Returns a torchvision transform that:
     Converts to a FloatTensor and scales pixel values [0,255] -> [0.0,1.0]
    """
    return transforms.Compose([
        transforms.ToTensor(),  # PIL [0–255] -> FloatTensor [0.0–1.0], shape C×H×W
    ])
IMG_TRANSFORM = get_default_transform()

## Run Your Policy

In [6]:
'''
Run one evaluation episode
'''
def run_one_episode():
    omy_env.reset()
    policy.reset()
    if IMAGE_NOISE:
        omy_env.agument_object_random_color()
    observation = omy_env.get_observation()
    omy_env.env.tick = 0
    while omy_env.env.is_viewer_alive() and omy_env.env.tick < MAX_EPISODE_STEPS:
        omy_env.step_env()
        if omy_env.env.loop_every(HZ = 20):
            success = omy_env.check_success()
            if success: break
            if omy_env.env.is_key_pressed_once(glfw.KEY_Z):
                break  # for debugging: press 'z' to end the episode
            agent_image, wrist_image = omy_env.grab_image(return_side=False)
            # # resize to 256x256
            frame = {
                "observation.state": observation,
            }
            if observation_type == 'image':
                agent_image = Image.fromarray(agent_image)
                wrist_image = Image.fromarray(wrist_image)
                agent_image = agent_image.resize((256, 256))
                wrist_image = wrist_image.resize((256, 256))
                agent_image = IMG_TRANSFORM(agent_image)
                wrist_image = IMG_TRANSFORM(wrist_image)
                frame["observation.image"] = agent_image
                frame["observation.wrist_image"] = wrist_image
            else:
                obj_states, recp_q_poses = omy_env.get_object_pose(pad=10)
                # extract position and rpy
                # sort the object names in alphabetical order to maintain consistency
                obj_states_sorted = np.zeros((24,), dtype=np.float32)
                sorted_indices = np.argsort(obj_states['names'])
                for i, idx in enumerate(sorted_indices):
                    if 'pad' in obj_states['names'][idx]:
                        continue
                    obj_state = obj_states['poses'][idx]
                    obj_states_sorted[i*6:(i+1)*6] = obj_state
                # sort the receptacle q states in alphabetical order
                recp_q_poses_sorted = np.zeros((3,), dtype=np.float32)
                sorted_indices = np.argsort(recp_q_poses['names'])
                i = 0
                for _, idx in enumerate(sorted_indices):
                    if 'pad' in recp_q_poses['names'][idx]:
                        continue
                    recp_q_pose = recp_q_poses['poses'][idx]
                    recp_q_poses_sorted[i] = recp_q_pose
                    i += 1
                obj_states_final = np.concatenate([obj_states_sorted, recp_q_poses_sorted])
                frame["observation.environment_state"] = obj_states_final
            # numpy to torch
            frame = {k: torch.tensor(v, dtype=torch.float32).unsqueeze(0).to(device) for k, v in frame.items()}

            action = policy.select_action(frame)
            action = action.squeeze(0).cpu().numpy()
            observation = omy_env.step(action, gripper_mode='continuous')
            omy_env.render()
    return success

In [7]:
'''
Run evaluation over multiple episodes
'''
results = []
for episode in range(TEST_EPISODES):
    success = run_one_episode()
    results.append(success)
    print(f"Episode {episode+1}/{TEST_EPISODES} - Success: {success}")
omy_env.env.close_viewer()
# log average success rate
avg_success = sum(results) / len(results)
print(f"Average Success Rate over {TEST_EPISODES} episodes: {avg_success*100:.2f}%")


['top', 'close']
DONE INITIALIZATION


/tmp/ipykernel_617464/2605272099.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  frame = {k: torch.tensor(v, dtype=torch.float32).unsqueeze(0).to(device) for k, v in frame.items()}


Episode 1/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 2/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 3/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 4/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 5/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 6/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 7/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 8/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 9/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 10/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 11/20 - Success: True
['top', 'close']
DONE INITIALIZATION
Episode 12/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 13/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 14/20 - Success: False
['top', 'close']
DONE INITIALIZATION
Episode 15/20 - Success: True
['top', 'close']
DONE INITIALIZA

### (Optional) Upload your model on huggingface

In [8]:
'''
Push to hub
Uncomment and fill in your HuggingFace username and repository name to push the model to the hub.
'''
# policy.push_to_hub(
#     repo_id='[Your_HuggingFace_Username]/[Your_Repository_Name]',
#     commit_message='Initial commit',
# )

'\nPush to hub\nUncomment and fill in your HuggingFace username and repository name to push the model to the hub.\n'